In [1]:
from glob import glob
import numpy as np
from parcels import FieldSet, ParticleSet, JITParticle
from parcels import ErrorCode, AdvectionRK4_3D

from datetime import timedelta
# from os import path
from datetime import datetime
# from parcels import ParcelsRandom
# import math
import xarray as xr

In [2]:
# Lorenz - MOi fields
data_path = '/storage/shared/oceanparcels/input_data/MOi/2019/'
output_path = '/storage/shared/oceanparcels/output_data/' + \
    'data_Claudio/backtrack_loc0_column.nc'

ufiles = sorted(glob(data_path + 'psy4v3r1-daily_U*.nc'))
vfiles = sorted(glob(data_path + 'psy4v3r1-daily_V*.nc'))
wfiles = sorted(glob(data_path + 'psy4v3r1-daily_W*.nc'))
tfiles = sorted(glob(data_path + 'psy4v3r1-daily_T_*.nc'))    
sfiles = sorted(glob(data_path + 'psy4v3r1-daily_S_*.nc'))  

mesh_mask = '/storage/shared/oceanparcels/input_data/MOi/domain_ORCA0083-N006/coordinates.nc'


In [9]:
filenames = {'U': {'lon': mesh_mask,
                   'lat': mesh_mask,
                   'depth': wfiles[0],
                   'data': ufiles},
             'V': {'lon': mesh_mask,
                   'lat': mesh_mask,
                   'depth': wfiles[0],
                   'data': vfiles},
             'W': {'lon': mesh_mask,
                   'lat': mesh_mask,
                   'depth': wfiles[0],
                   'data': wfiles}}

filenames['temperature'] = {'lon': mesh_mask, 
                                 'lat': mesh_mask, 
                                 'depth': wfiles[0], 
                                 'data': tfiles}
filenames['salinity'] = {'lon': mesh_mask, 
                             'lat': mesh_mask, 
                             'depth': wfiles[0], 
                             'data': sfiles}

In [10]:
n_points = 1000
# start_time = datetime.strptime('2007-08-22 12:00:00', '%Y-%m-%d %H:%M:%S')

# start_time = datetime.strptime('2010-12-20 12:00:00', '%Y-%m-%d %H:%M:%S')
start_time = datetime.strptime('2019-12-02 12:00:00', '%Y-%m-%d %H:%M:%S')
# psy4v3r1-daily_2D_2019-01-01.nc

variables = {'U': 'vozocrtx',
             'V': 'vomecrty',
             'W': 'vovecrtz'}

variables['temperature'] = 'votemper'
variables['salinity'] = 'vosaline'

dimensions = {'U': {'lon': 'glamf',
                    'lat': 'gphif',
                    'depth': 'depthw',
                    'time': 'time_counter'},
              'V': {'lon': 'glamf',
                    'lat': 'gphif',
                    'depth': 'depthw',
                    'time': 'time_counter'},
              'W': {'lon': 'glamf',
                    'lat': 'gphif',
                    'depth': 'depthw',
                    'time': 'time_counter'}}

dimensions['temperature'] = {'lon': 'glamf', 
                                  'lat': 'gphif',
                                  'depth': 'depthw', 
                                  'time': 'time_counter'}

dimensions['salinity'] = {'lon': 'glamf', 
                              'lat': 'gphif',
                              'depth': 'depthw', 
                              'time': 'time_counter'}

indices = {'lat': range(750, 1300), 'lon': range(2900, 4000)}
# indices = {'lat': range(500, 1400), 'lon': range(2500, 3800)}

In [11]:
fieldset = FieldSet.from_nemo(filenames, variables, dimensions,
                              allow_time_extrapolation=False,
                              indices=indices)

In [12]:
lon_cluster = [6.287]*n_points
lat_cluster = [-32.171]*n_points
# depth_cluster = [5000]*n_points  # closest level to -5000m
depth_cluster = np.linspace(1, 5000, n_points)

date_cluster = [start_time]*n_points

# lon_cluster = np.array(lon_cluster)+(np.random.random(len(lon_cluster))-0.5)/12
# lat_cluster = np.array(lat_cluster)+(np.random.random(len(lat_cluster))-0.5)/12

In [13]:
pset = ParticleSet.from_list(fieldset=fieldset, pclass=JITParticle,
                             lon=lon_cluster,
                             lat=lat_cluster,
                             depth=depth_cluster,
                             time=date_cluster)


In [14]:
def delete_particle(particle, fieldset, time):
    particle.delete()

In [15]:
kernels = pset.Kernel(AdvectionRK4_3D)

# Output file
output_file = pset.ParticleFile(name=output_path,
                                outputdt=timedelta(hours=24))

pset.execute(kernels,
             output_file=output_file,
             runtime=timedelta(days=10),
             dt=-timedelta(hours=1),
             recovery={ErrorCode.ErrorOutOfBounds: delete_particle})

output_file.close()


sh: None: command not found
INFO: Compiled ArrayJITParticleAdvectionRK4_3D ==> /tmp/parcels-263482/lib61d26c0656da65de5f1793e733a9ad34_0.so
INFO: Temporary output files are stored in /storage/shared/oceanparcels/output_data/data_Claudio/out-CNRTLODL.
INFO: You can use "parcels_convert_npydir_to_netcdf /storage/shared/oceanparcels/output_data/data_Claudio/out-CNRTLODL" to convert these to a NetCDF file during the run.
100% (864000.0 of 864000.0) |############| Elapsed Time: 0:02:04 Time:  0:02:04


In [16]:
sim = xr.load_dataset(output_path)

In [17]:
sim

<xarray.Dataset>
Dimensions:     (traj: 1000, obs: 11)
Dimensions without coordinates: traj, obs
Data variables:
    trajectory  (traj, obs) float64 0.0 0.0 0.0 0.0 ... 999.0 999.0 999.0 999.0
    time        (traj, obs) datetime64[ns] 2019-12-02T12:00:00 ... 2019-11-22...
    lat         (traj, obs) float64 -32.17 -32.12 -32.07 ... -32.34 -32.34
    lon         (traj, obs) float64 6.287 6.459 6.649 6.89 ... 7.11 7.149 7.186
    z           (traj, obs) float64 1.0 0.9974 0.9571 ... 4.994e+03 4.993e+03
Attributes:
    feature_type:           trajectory
    Conventions:            CF-1.6/CF-1.7
    ncei_template_version:  NCEI_NetCDF_Trajectory_Template_v2.0
    parcels_version:        v2.3.0-40-g6423cc0e
    parcels_mesh:           spherical